### Importing Libraries

In [39]:
import mysql.connector
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings('ignore')

### Connecting Python with MySQL server

In [4]:
conn = mysql.connector.connect(
    host='localhost',
    user='root',
    password='',
)

In [5]:
cursor = conn.cursor()

In [6]:
cursor.execute("CREATE DATABASE my_db")

In [7]:
cursor.execute("USE my_db")

In [8]:
conn.database

'my_db'

### Inserting a pandas dataframe into MySQL table

In [9]:
diamond_data = pd.read_csv('diamonds.csv')

In [11]:
create_table_query = "CREATE TABLE diamond ("
create_table_query += ", " .join([f"`{col}` VARCHAR(255)" for col in diamond_data.columns])
create_table_query += ")"

cursor.execute(create_table_query)

In [12]:
for i,row in diamond_data.iterrows():
    insert_query = "INSERT INTO diamond VALUES (" + ", ".join(['%s'] * len(diamond_data.columns)) + ")"
    cursor.execute(insert_query, tuple(row))

In [13]:
conn.commit()

### Extracting data from MySQL database into a pandas dataframe for data preprocessing & ML

In [15]:
query = "SELECT * FROM diamond"
result_data = pd.read_sql(query, conn)

In [16]:
result_data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.2,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


### Data Preprocessing

In [17]:
result_data.dtypes

carat      object
cut        object
color      object
clarity    object
depth      object
table      object
price      object
x          object
y          object
z          object
dtype: object

In [23]:
columns = ['carat', 'depth', 'table', 'price', 'x', 'y', 'z']

for column in columns:
    result_data[column] = result_data[column].astype(float)

In [24]:
result_data.dtypes

carat      float64
cut         object
color       object
clarity     object
depth      float64
table      float64
price      float64
x          float64
y          float64
z          float64
dtype: object

In [26]:
label_encoder = LabelEncoder()
cols = ['cut', 'color', 'clarity']

for col in cols:
    result_data[col] = label_encoder.fit_transform(result_data[col])

In [27]:
result_data.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,2,1,3,61.5,55.0,326.0,3.95,3.98,2.43
1,0.21,3,1,2,59.8,61.0,326.0,3.89,3.84,2.31
2,0.23,1,1,4,56.9,65.0,327.0,4.05,4.07,2.31
3,0.29,3,5,5,62.4,58.0,334.0,4.20,4.23,2.63
4,0.31,1,6,3,63.3,58.0,335.0,4.34,4.35,2.75


In [28]:
result_data.corr()

,carat,cut,color,clarity,depth,table,price,x,y,z
carat,1.000000,0.017124,0.291437,-0.214290,0.028224,0.181618,0.921591,0.975094,0.951722,0.953387
cut,0.017124,1.000000,0.000304,0.028235,-0.194249,0.150327,0.039860,0.022342,0.027572,0.002037
color,0.291437,0.000304,1.000000,-0.027795,0.047279,0.026465,0.172511,0.270287,0.263584,0.268227
clarity,-0.214290,0.028235,-0.027795,1.000000,-0.053080,-0.088223,-0.071535,-0.225721,-0.217616,-0.224263
depth,0.028224,-0.194249,0.047279,-0.053080,1.000000,-0.295779,-0.010647,-0.025289,-0.029341,0.094924
table,0.181618,0.150327,0.026465,-0.088223,-0.295779,1.000000,0.127134,0.195344,0.183760,0.150929
price,0.921591,0.039860,0.172511,-0.071535,-0.010647,0.127134,1.000000,0.884435,0.865421,0.861249
x,0.975094,0.022342,0.270287,-0.225721,-0.025289,0.195344,0.884435,1.000000,0.974701,0.970772
y,0.951722,0.027572,0.263584,-0.217616,-0.029341,0.183760,0.865421,0.974701,1.000000,0.952006
z,0.953387,0.002037,0.268227,-0.224263,0.094924,0.150929,0.861249,0.970772,0.952006,1.000000


In [31]:
result_data.shape

(53940, 10)

### Applying Multivariate Linear Regression on the extracted data

In [30]:
independent_variables = ['carat', 'x', 'y', 'z'] # highly correlated variables
X = result_data[independent_variables]
y = result_data['price']

In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
# building the model class 

class MultiLinearRegression:
    def __init__(self, n_iters, learning_rate):
        self.learning_rate = learning_rate # controls step size during gradient descent
        self.n_iters = n_iters # no. of iterations/steps for gradient descent
        self.coeff = None # coefficient
        self.inter = None # intercept

    # training the model
    def fit(self, X, y):
        n_samples, n_features = X.shape 

        self.coeff = np.zeros(n_features) # initializing coefficients to 0
        self.inter = 0 # initializing intercept to 0

        for _ in range(self.n_iters):

            y_pred = np.dot(X, self.coeff) + self.inter # calculates the predicted values using current m and c

            # calculate gradient/derivatives for coefficient
            dm = (1 / n_samples) * np.dot(X.T, (y_pred - y)) 
            # calculate gradient/derivative for intercept
            dc = (1 / n_samples) * np.sum(y_pred - y) 
            
            # update coefficient using gradient descent by subtracting product of gradients and learning rate
            self.coeff = self.coeff - self.learning_rate * dm 
            # update intercept using gradient descent by subtracting product of gradients and learning rate
            self.inter = self.inter - self.learning_rate * dc 
    
    # model prediction
    def predict(self, X):
        # dot product between input X and learned coefficient and then adding by learned intercept
        return np.dot(X, self.coeff) + self.inter

In [58]:
regressor = MultiLinearRegression(n_iters=30000, learning_rate=0.01)

In [59]:
regressor.fit(X_train, y_train)

In [60]:
y_pred_train = regressor.predict(X_train)

In [61]:
# training accuracy

R2_train = r2_score(y_train, y_pred_train)
R2

0.8491278033356742

In [62]:
y_pred_test = regressor.predict(X_test)

In [63]:
# testing accuracy

R2_test = r2_score(y_test, y_pred_test)
R2_test

0.8525282940378436